In [41]:

import numpy as np
import pandas as pd

In [42]:
match = pd.read_csv('ODI_Match_info.csv')
delivery = pd.read_csv('ODI_Match_Data.csv', low_memory=False)

In [43]:
match.head()


,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1389389,2023/24,Indore,2023/09/24,India,Australia,Australia,field,D/L,1,India,99,0,SS Iyer,"Holkar Cricket Stadium, Indore",J Madanagopal,HDPK Dharmasena,KN Ananthapadmanabhan
1,1336129,2023,Nottingham,2023/09/23,England,Ireland,Ireland,field,normal,0,England,48,0,WG Jacks,"Trent Bridge, Nottingham",DJ Millns,RJ Tucker,PR Reiffel
2,1395701,2023,Dhaka,2023/09/23,New Zealand,Bangladesh,New Zealand,bat,normal,0,New Zealand,86,0,IS Sodhi,"Shere Bangla National Stadium, Mirpur",M Erasmus,Sharfuddoula,Nitin Menon
3,1389388,2023/24,Chandigarh,2023/09/22,Australia,India,India,field,normal,0,India,0,5,Mohammed Shami,"Punjab Cricket Association IS Bindra Stadium, ...",KN Ananthapadmanabhan,HDPK Dharmasena,J Madanagopal
4,1395700,2023,Dhaka,2023/09/21,New Zealand,Bangladesh,Bangladesh,field,normal,0,NaN,0,0,NaN,"Shere Bangla National Stadium, Mirpur",Nitin Menon,Sharfuddoula,M Erasmus


In [44]:
match.shape

(1973, 18)

In [45]:
delivery.head()

,match_id,season,start_date,venue,innings,ball,over,ballofover,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,total_runs,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,0,1.0,India,Australia,RD Gaikwad,Shubman Gill,SH Johnson,4,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1389389
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,0,2.0,India,Australia,RD Gaikwad,Shubman Gill,SH Johnson,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1389389
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,0,3.0,India,Australia,RD Gaikwad,Shubman Gill,SH Johnson,4,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1389389
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,0,4.0,India,Australia,RD Gaikwad,Shubman Gill,SH Johnson,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1389389
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,0,5.0,India,Australia,RD Gaikwad,Shubman Gill,SH Johnson,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1389389


In [46]:
delivery.shape

(1048355, 26)

In [47]:
finalscoredf = delivery.groupby(['match_id', 'innings']).sum()['total_runs'].reset_index()

In [48]:
finalscoredf =finalscoredf[finalscoredf['innings']==1]

In [49]:

finalscoredf

,match_id,innings,total_runs
0,247457,1,241
2,247458,1,334
4,247460,1,321
6,247461,1,221
8,247462,1,353
...,...,...,...
3890,1390344,1,201
3892,1390345,1,300
3894,1390346,1,268
3896,1395700,1,136


In [50]:
matchdf = match.merge(finalscoredf[['match_id', 'total_runs']], left_on='id', right_on='match_id')

In [51]:
teams= [
    'India',
    'England', 
    'New Zealand', 
    'Australia', 
    'Sri Lanka',
    'South Africa', 
    'Bangladesh', 
    'Pakistan', 
    'Nepal', 
    'Afghanistan',
    'West Indies', 
    'Scotland', 
    'United Arab Emirates', 
    'Oman',
    'Netherlands', 
    'Zimbabwe', 
    'United States of America', 
    'Ireland',
    'Canada', 
    'Namibia',
    'Papua New Guinea', 
    'Hong Kong',
    'Kenya', 
]

In [52]:
matchdf = matchdf[matchdf['team1'].isin(teams)]
matchdf = matchdf[matchdf['team2'].isin(teams)]

In [53]:
matchdf = matchdf[matchdf['dl_applied']==0]

In [54]:
matchdf = matchdf[['match_id','city','winner','total_runs']]

In [55]:
deliverydf = matchdf.merge(delivery,on='match_id')

In [56]:
deliverydf = deliverydf[deliverydf['innings'] == 2]

In [57]:
pd.set_option('display.max_columns', None)
deliverydf=deliverydf.dropna(subset=['city','winner'])

In [58]:
print(deliverydf['total_runs_y'].dtype)
deliverydf['currentscore'] = deliverydf.groupby('match_id')['total_runs_y'].cumsum()


int64


In [59]:
deliverydf['runs_left'] = deliverydf['total_runs_x'] - deliverydf['currentscore'] + 1

In [60]:
deliverydf['balls_left'] = 300 - (deliverydf['over']*6 + deliverydf['ballofover'])
deliverydf

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,over,ballofover,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,total_runs_y,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,currentscore,runs_left,balls_left
310,1336129,Nottingham,England,334,2023,2023-09-23,"Trent Bridge, Nottingham",2,0.1,0,1.0,Ireland,England,A Balbirnie,PR Stirling,MJ Potts,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1336129,0,335,299.0
311,1336129,Nottingham,England,334,2023,2023-09-23,"Trent Bridge, Nottingham",2,0.2,0,2.0,Ireland,England,A Balbirnie,PR Stirling,MJ Potts,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1336129,1,334,298.0
312,1336129,Nottingham,England,334,2023,2023-09-23,"Trent Bridge, Nottingham",2,0.3,0,3.0,Ireland,England,PR Stirling,A Balbirnie,MJ Potts,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1336129,1,334,297.0
313,1336129,Nottingham,England,334,2023,2023-09-23,"Trent Bridge, Nottingham",2,0.4,0,4.0,Ireland,England,PR Stirling,A Balbirnie,MJ Potts,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1336129,1,334,296.0
314,1336129,Nottingham,England,334,2023,2023-09-23,"Trent Bridge, Nottingham",2,0.5,0,5.0,Ireland,England,PR Stirling,A Balbirnie,MJ Potts,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1336129,1,334,295.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957650,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.3,39,3.0,South Africa,Pakistan,M Ntini,SM Pollock,Shahid Afridi,0,0,0,NaN,NaN,NaN,NaN,NaN,caught,M Ntini,NaN,NaN,250669,208,144,63.0
957651,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.4,39,4.0,South Africa,Pakistan,SM Pollock,CK Langeveldt,Shahid Afridi,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250669,208,144,62.0
957652,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.5,39,5.0,South Africa,Pakistan,SM Pollock,CK Langeveldt,Shahid Afridi,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250669,209,143,61.0
957653,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.6,39,6.0,South Africa,Pakistan,CK Langeveldt,SM Pollock,Shahid Afridi,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250669,209,143,60.0


In [61]:
deliverydf['player_dismissed'] = deliverydf['player_dismissed'].fillna("0")


In [62]:
deliverydf['player_dismissed'] = deliverydf['player_dismissed'].apply(lambda x:x if x == "0" else "1")


In [63]:
deliverydf['player_dismissed'] = deliverydf['player_dismissed'].astype('int')
deliverydf

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,over,ballofover,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,total_runs_y,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,currentscore,runs_left,balls_left
310,1336129,Nottingham,England,334,2023,2023-09-23,"Trent Bridge, Nottingham",2,0.1,0,1.0,Ireland,England,A Balbirnie,PR Stirling,MJ Potts,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1336129,0,335,299.0
311,1336129,Nottingham,England,334,2023,2023-09-23,"Trent Bridge, Nottingham",2,0.2,0,2.0,Ireland,England,A Balbirnie,PR Stirling,MJ Potts,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1336129,1,334,298.0
312,1336129,Nottingham,England,334,2023,2023-09-23,"Trent Bridge, Nottingham",2,0.3,0,3.0,Ireland,England,PR Stirling,A Balbirnie,MJ Potts,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1336129,1,334,297.0
313,1336129,Nottingham,England,334,2023,2023-09-23,"Trent Bridge, Nottingham",2,0.4,0,4.0,Ireland,England,PR Stirling,A Balbirnie,MJ Potts,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1336129,1,334,296.0
314,1336129,Nottingham,England,334,2023,2023-09-23,"Trent Bridge, Nottingham",2,0.5,0,5.0,Ireland,England,PR Stirling,A Balbirnie,MJ Potts,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1336129,1,334,295.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957650,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.3,39,3.0,South Africa,Pakistan,M Ntini,SM Pollock,Shahid Afridi,0,0,0,NaN,NaN,NaN,NaN,NaN,caught,1,NaN,NaN,250669,208,144,63.0
957651,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.4,39,4.0,South Africa,Pakistan,SM Pollock,CK Langeveldt,Shahid Afridi,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,250669,208,144,62.0
957652,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.5,39,5.0,South Africa,Pakistan,SM Pollock,CK Langeveldt,Shahid Afridi,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,250669,209,143,61.0
957653,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.6,39,6.0,South Africa,Pakistan,CK Langeveldt,SM Pollock,Shahid Afridi,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,250669,209,143,60.0


In [64]:
wickets = deliverydf.groupby('match_id')['player_dismissed'].cumsum()
deliverydf['wickets_left'] = 10 - wickets
deliverydf.tail(6)

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,over,ballofover,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,total_runs_y,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,currentscore,runs_left,balls_left,wickets_left
957649,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.2,39,2.0,South Africa,Pakistan,M Ntini,SM Pollock,Shahid Afridi,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,250669,208,144,64.0,2
957650,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.3,39,3.0,South Africa,Pakistan,M Ntini,SM Pollock,Shahid Afridi,0,0,0,NaN,NaN,NaN,NaN,NaN,caught,1,NaN,NaN,250669,208,144,63.0,1
957651,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.4,39,4.0,South Africa,Pakistan,SM Pollock,CK Langeveldt,Shahid Afridi,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,250669,208,144,62.0,1
957652,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.5,39,5.0,South Africa,Pakistan,SM Pollock,CK Langeveldt,Shahid Afridi,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,250669,209,143,61.0,1
957653,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,39.6,39,6.0,South Africa,Pakistan,CK Langeveldt,SM Pollock,Shahid Afridi,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,250669,209,143,60.0,1
957654,250669,Durban,Pakistan,351,2006/07,2007-02-07,Kingsmead,2,40.1,40,1.0,South Africa,Pakistan,SM Pollock,CK Langeveldt,Shoaib Malik,0,1,1,1.0,NaN,NaN,NaN,NaN,stumped,1,NaN,NaN,250669,210,142,59.0,0


In [65]:
deliverydf['crr'] = (deliverydf['currentscore']*6)/(300 - deliverydf['balls_left'])

In [66]:
deliverydf['rrr'] = (deliverydf['runs_left']*6)/deliverydf['balls_left']

In [67]:
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

deliverydf['result'] = deliverydf.apply(result,axis=1)

In [68]:
finaldf = deliverydf[['batting_team','bowling_team','city','runs_left','balls_left','wickets_left','total_runs_x','crr','rrr','result']]


In [69]:
finaldf = finaldf.sample(finaldf.shape[0])

finaldf

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,crr,rrr,result
201636,Sri Lanka,West Indies,Colombo,75,71.0,4,289,5.633188,6.338028,1
45014,Namibia,United Arab Emirates,Windhoek,169,175.0,7,267,4.752000,5.794286,0
534145,Sri Lanka,India,Cuttack,315,233.0,9,363,4.388060,8.111588,0
52628,Netherlands,Zimbabwe,Harare,39,25.0,4,249,4.603636,9.360000,1
180395,England,India,Pune,174,155.0,5,329,6.455172,6.735484,0
...,...,...,...,...,...,...,...,...,...,...
676415,Sri Lanka,South Africa,Paarl,271,221.0,3,301,2.354430,7.357466,0
71013,India,Sri Lanka,Kolkata,57,101.0,6,215,4.793970,3.386139,1
591586,India,Australia,Jaipur,74,74.0,9,359,7.592920,6.000000,1
434002,England,Sri Lanka,Birmingham,166,204.0,10,254,5.562500,4.882353,1


In [70]:

finaldf.dropna(inplace=True)
finaldf=finaldf[finaldf['balls_left']!=0]
finaldf.sample()

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,crr,rrr,result
620212,Pakistan,South Africa,Johannesburg,314,265.0,9,343,5.142857,7.109434,0


In [71]:
X = finaldf.iloc[:,:-1]
y = finaldf.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [72]:
X_train

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,crr,rrr
205683,Oman,United States of America,Kirtipur,212,297.0,10,213,4.000000,4.282828
280366,South Africa,Pakistan,Durban,136,225.0,7,203,5.440000,3.626667
137127,Papua New Guinea,United Arab Emirates,Dubai,47,139.0,7,147,3.763975,2.028777
499242,Ireland,United Arab Emirates,Brisbane,272,277.0,9,278,1.826087,5.891697
829885,Bangladesh,West Indies,St Kitts,220,284.0,10,248,10.875000,4.647887
...,...,...,...,...,...,...,...,...,...
553639,Sri Lanka,South Africa,Colombo,167,154.0,7,304,5.671233,6.506494
32565,United Arab Emirates,Sri Lanka,Bulawayo,276,207.0,9,355,5.161290,8.000000
859237,West Indies,Pakistan,Abu Dhabi,211,254.0,8,232,2.869565,4.984252
413408,India,New Zealand,Delhi,170,184.0,6,242,3.775862,5.543478


In [73]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse_output=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [75]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',RandomForestClassifier())
])

In [76]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', RandomForestClassifier())])

In [77]:
y_pred = pipe.predict(X_test)

In [78]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9998793436293436

In [79]:
pipe.predict_proba[8]

TypeError: 'method' object is not subscriptable